In [1]:
import os
import requests
from datetime import datetime
import re
import json  # Para exportar el resumen
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numpy as np


C:\Users\FERNANDO\AppData\Local\Temp\ipykernel_21748\4042661585.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
inicio="1990" # inicio
fin="2024" # termina en 2024 todas las series 
fechas = pd.date_range(start=inicio, end=fin, freq='YS') # creamos las fechas para que tengan solo mes y año
fechas_lista = fechas.strftime('%Y').tolist() # ACA ESTAN LAS FECHAS
print(fechas_lista)

['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


### DEFLACTOR DEL PBI
---
SERIE REAL : PM04860AA
SERIE NOMINAL : PM04946AA

In [3]:
# ESTA PRIMERA CELDA ES UNA MUESTRA DE LO QUE SE HACE
codigo="PM04860AA" 
url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/1990/2024' # las fechas de inicio y final tmbn pueden ser delimitadas
response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
data_real=response.json() # lo almacenamos en un elemenot llamado data
print(data_real) # la data es un diccionario que contiene todos los valores que necesitamos

{'config': {'title': 'Producto bruto interno y otros indicadores', 'series': [{'name': 'Producto bruto interno y otros indicadores - PBI (millones S/ 2007)', 'dec': '0'}]}, 'periods': [{'name': '1990', 'values': ['151492']}, {'name': '1991', 'values': ['154854']}, {'name': '1992', 'values': ['154017']}, {'name': '1993', 'values': ['162093']}, {'name': '1994', 'values': ['182043.671320207']}, {'name': '1995', 'values': ['195535.99143694']}, {'name': '1996', 'values': ['201009.289724016']}, {'name': '1997', 'values': ['214028.291389057']}, {'name': '1998', 'values': ['213189.993392589']}, {'name': '1999', 'values': ['216376.740630514']}, {'name': '2000', 'values': ['222206.707348694']}, {'name': '2001', 'values': ['223579.575750618']}, {'name': '2002', 'values': ['235772.947128979']}, {'name': '2003', 'values': ['245592.61375299']}, {'name': '2004', 'values': ['257769.786233073']}, {'name': '2005', 'values': ['273971.153886791']}, {'name': '2006', 'values': ['294597.830810429']}, {'name'

In [4]:
# ESTA PRIMERA CELDA ES UNA MUESTRA DE LO QUE SE HACE
codigo="PM04946AA" 
url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/1990/2024' # las fechas de inicio y final tmbn pueden ser delimitadas
response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
data_nominal=response.json() # lo almacenamos en un elemenot llamado data
print(data_nominal) # la data es un diccionario que contiene todos los valores que necesitamos

{'config': {'title': 'Producto bruto interno por tipo de gasto (millones S/)', 'series': [{'name': 'Producto bruto interno por tipo de gasto (millones S/) - PBI', 'dec': '0'}]}, 'periods': [{'name': '1990', 'values': ['5321.98244209542']}, {'name': '1991', 'values': ['26256.239275889']}, {'name': '1992', 'values': ['43990.2193378417']}, {'name': '1993', 'values': ['68078.5757586313']}, {'name': '1994', 'values': ['94724.4534310858']}, {'name': '1995', 'values': ['115571.292007033']}, {'name': '1996', 'values': ['130810.705956434']}, {'name': '1997', 'values': ['149749.962850056']}, {'name': '1998', 'values': ['157833.56802713']}, {'name': '1999', 'values': ['164770.962276773']}, {'name': '2000', 'values': ['175862.061329436']}, {'name': '2001', 'values': ['178974.626321977']}, {'name': '2002', 'values': ['189741.447349983']}, {'name': '2003', 'values': ['203612.612660881']}, {'name': '2004', 'values': ['225691.818733597']}, {'name': '2005', 'values': ['244651.600262466']}, {'name': '20

In [5]:
datos_real=[float(elem['values'][0]) for elem in data_real['periods']]
print(datos_real)

datos_nominal=[float(elem['values'][0]) for elem in data_nominal['periods']]
print(datos_nominal)

deflactor=[]

for nominal, real in zip(datos_nominal, datos_real):
    if real != 0:
        valor_deflactor = (nominal / real) * 100
    else:
        valor_deflactor = None  # o podrías usar 0, np.nan, etc.
    deflactor.append(valor_deflactor)

print(deflactor)



[151492.0, 154854.0, 154017.0, 162093.0, 182043.671320207, 195535.99143694, 201009.289724016, 214028.291389057, 213189.993392589, 216376.740630514, 222206.707348694, 223579.575750618, 235772.947128979, 245592.61375299, 257769.786233073, 273971.153886791, 294597.830810429, 319693.0, 348870.0, 352693.0, 382081.0, 406256.0, 431199.0, 456435.0, 467308.0, 482506.0, 501581.0, 514215.0, 534626.0, 546605.0, 487191.0, 552285.835041041, 567798.450475716, 565508.784042891, 584357.549500324]
[5321.98244209542, 26256.239275889, 43990.2193378417, 68078.5757586313, 94724.4534310858, 115571.292007033, 130810.705956434, 149749.962850056, 157833.56802713, 164770.962276773, 175862.061329436, 178974.626321977, 189741.447349983, 203612.612660881, 225691.818733597, 244651.600262466, 286314.083739823, 319693.0, 354827.53577412, 367052.845311735, 421582.065233283, 471985.536794554, 509758.201757533, 548696.515475435, 578493.241064411, 614251.096701769, 662788.39452977, 706232.91490069, 748094.97252289, 777524

In [9]:
columnas = ['Fecha','PBI_real','PBI_nominal','DEFLACTOR']

datos=[fechas_lista]+[datos_real]+[datos_nominal]+[deflactor]

dic_data={}

for i in range(len(datos)): # se arma el diccionario con la data correspondiente a cada columna
    dic_data[columnas[i]]=datos[i]


df=pd.DataFrame(dic_data,columns=columnas)
df
directorio='C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/outputs'
os.chdir(directorio)
df.to_excel('DEFLACTOR.xlsx', index=False)
df

,Fecha,PBI_real,PBI_nominal,DEFLACTOR
0,1990,151492.000000,5.321982e+03,3.513045
1,1991,154854.000000,2.625624e+04,16.955480
2,1992,154017.000000,4.399022e+04,28.561925
3,1993,162093.000000,6.807858e+04,41.999701
4,1994,182043.671320,9.472445e+04,52.033917
5,1995,195535.991437,1.155713e+05,59.104869
6,1996,201009.289724,1.308107e+05,65.076946
7,1997,214028.291389,1.497500e+05,69.967368
8,1998,213189.993393,1.578336e+05,74.034229
9,1999,216376.740631,1.647710e+05,76.150034
